In [2]:
import pandas as pd

In [4]:

df=pd.read_csv('train.csv')

In [6]:

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
###Drop Nan Values
df=df.dropna()

In [8]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [9]:
## Get the Dependent features
y=df['label']

In [10]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [11]:
X.shape,y.shape

((18285, 4), (18285,))

In [12]:
import tensorflow as tf
tf.__version__

'2.3.1'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [14]:
### Vocabulary size
voc_size=5000

In [15]:
messages=X.copy()

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\DELL
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3720 3076 3421]
 [   0    0    0 ... 2803 2511 4092]
 [   0    0    0 ... 3316 2047 1771]
 ...
 [   0    0    0 ...  354 1432 1912]
 [   0    0    0 ... 4330 3171 1239]
 [   0    0    0 ... 1396 2191  341]]


In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1718,
       4603, 2043, 4671, 2567, 3311, 1086, 3720, 3076, 3421])

In [25]:

## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
len(embedded_docs),y.shape

(18285, (18285,))

In [28]:

import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [31]:

### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 60ms/step - loss: 0.2887 - accuracy: 0.8656 - val_loss: 0.2060 - val_accuracy: 0.9137
Epoch 2/10
192/192 [==============================] - 9s 45ms/step - loss: 0.1417 - accuracy: 0.9443 - val_loss: 0.2295 - val_accuracy: 0.9099
Epoch 3/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0978 - accuracy: 0.9635 - val_loss: 0.2496 - val_accuracy: 0.9074
Epoch 4/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0767 - accuracy: 0.9719 - val_loss: 0.3400 - val_accuracy: 0.9044
Epoch 5/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0546 - accuracy: 0.9809 - val_loss: 0.3201 - val_accuracy: 0.9110
Epoch 6/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0306 - accuracy: 0.9889 - val_loss: 0.4543 - val_accuracy: 0.9041
Epoch 7/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0245 - accuracy: 0.9910 - val_loss: 0.4712 - val_accuracy: 0.9

In [32]:
y_pred1=model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(y_test,y_pred1)

array([[3105,  314],
       [ 267, 2349]], dtype=int64)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.903728251864126

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      3419
           1       0.88      0.90      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

